In [1]:
import pandas as pd
import glob
import json

# For Location Extraction
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

In [2]:
# For Extracting Location from Text

# Loading the Stanford jar file
import os
java_path = "C:/Program Files/Java/jdk-14.0.2/bin/java.exe"
os.environ['JAVAHOME'] = java_path

st = StanfordNERTagger('./stanford-ner-4.0.0/classifiers/english.all.3class.distsim.crf.ser.gz', './stanford-ner-4.0.0/stanford-ner.jar', encoding='utf-8')
 
def location_from_text(text):
    print(text)
    locations = ''
    temp = [ i.capitalize() for i in text.split(' ')]
    text = ''
    for i in temp:
        text += (i + ' ')
        
    text = text[:-1]

    tokenized_text = word_tokenize(text)
    classified_text = st.tag(tokenized_text)
    
    for i in classified_text:
        if i[1] == 'LOCATION':
            locations += i[0] + ","
            
    return locations[:-1]

In [3]:
folders = glob.glob("Data/*")

In [4]:
dictlist = []

for folder in folders:
    if folder == "Data/user":
        continue
    files = glob.glob(folder + "/tweets/*")
    for file in files:
        json_string = open(file,'r',encoding="utf8").read()
        json_dict = json.loads(json_string)
        json_dict["search_term"] = folder[5:].capitalize()
        dictlist.append(json_dict)

In [5]:
df = pd.DataFrame(dictlist)
df = df.replace({'\n': ' '}, regex=True) # remove linebreaks in the dataframe
df = df.replace({'\t': ' '}, regex=True) # remove tabs in the dataframe
df = df.replace({'\r': ' '}, regex=True) # remove carriage return in the dataframe

In [6]:
df.sort_values("datetime",ascending = True,inplace=True,na_position="last")

In [7]:
df

,usernameTweet,ID,text,url,nbr_retweet,nbr_favorite,nbr_reply,datetime,is_reply,is_retweet,user_id,search_term,has_media,medias
25862,quake08,1184286008,[blog] Dam May Have Triggered Massive China ...,/quake08/status/1184286008,0,0,0,2009-02-07 01:27:50,False,False,14811425,Zipingpu dam,NaN,NaN
25863,BillGlover,1186041453,Did The Zipingpu Dam Cause The Sichuan Ear...,/BillGlover/status/1186041453,0,0,0,2009-02-07 16:26:22,False,False,15672198,Zipingpu dam,NaN,NaN
25864,quake08,1188282166,[blog] NAACAL: Chinese earthquake may have bee...,/quake08/status/1188282166,0,0,0,2009-02-08 11:56:44,False,False,14811425,Zipingpu dam,NaN,NaN
25865,elpanzer_,1188379203,Did The Zipingpu Dam Cause The Sichuan Ear...,/elpanzer_/status/1188379203,0,0,0,2009-02-08 13:20:36,False,False,11744492,Zipingpu dam,NaN,NaN
25866,quake08,1191165662,[blog] Could The Zipingpu Dam Have Caused ...,/quake08/status/1191165662,0,0,0,2009-02-09 12:26:27,False,False,14811425,Zipingpu dam,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12471,GleyberGotNext,1297746041022767105,We were awful . But the Nuggets game told me ...,/GleyberGotNext/status/1297746041022767105,0,0,0,2020-08-24 09:32:21,True,False,1279831065071202305,Awful,NaN,NaN
12473,rhursday,1297746045317918721,Donald Trump makes my blood boil and knowing t...,/rhursday/status/1297746045317918721,0,0,1,2020-08-24 09:32:22,False,False,2502829208,Awful,NaN,NaN
13678,emndo77,1297746044071972864,お手数おかけして申し訳ないです,/emndo77/status/1297746044071972864,0,0,0,2020-08-24 09:32:22,True,False,1289677921024536576,Cactus,NaN,NaN
12472,hs1fearless,1297746041949782017,i’m awful ok,/hs1fearless/status/1297746041949782017,0,0,0,2020-08-24 09:32:22,False,False,1234336014195445760,Awful,NaN,NaN


In [8]:
df["location_from_text"] = df["text"].apply(location_from_text , 1)
df

UnboundLocalError: local variable 'i' referenced before assignment

In [8]:
with open("CollectedData.csv" , "w", encoding="utf-8") as f:
    df.to_csv(f,index=False)

In [ ]:
import requests , bs4,urllib
df = pd.DataFrame(pd.read_csv("CollectedData.csv"))
image_df = df[["ID","has_media","medias","search_term"]]
image_df.dropna(inplace=True)
indexes = image_df[image_df["medias"] == "[]"].index
image_df.drop(indexes,inplace=True)
image_df.head()

In [18]:
import mimetypes,os,os.path

def is_url_image(url):    
    mimetype,encoding = mimetypes.guess_type(url)
    print((mimetype and mimetype.startswith('image')))
    return (mimetype and mimetype.startswith('image'))

In [19]:
def download_image(item):
    print(item)
    import requests
    import time
    if item["medias"] == None:
        return 
    item["medias"] = item["medias"].strip("][")
    item["medias"] = item["medias"].strip("\'")
    
    r = requests.get(item["medias"], stream=True)
    if os.path.isdir("Data/{}/content".format(item["search_term"])) == False:
        os.mkdir("Data/{}/content".format(item["search_term"]))
    ext = r.headers['content-type'].split('/')[-1]
    ext = ext.split(";")[0] # converts response headers mime type to an extension (may not work with everything)
    with open("Data/{}/content/{}.{}".format(item["search_term"],item["ID"],ext), 'wb') as f: # open the file to write as binary - replace 'wb' with 'w' for text files
        for chunk in r.iter_content(1024): # iterate on stream using 1KB packets
            f.write(chunk) # write the file


image_df = image_df.apply(download_image , 1)

ID                     1010193501798191104
has_media                             True
medias         ['https://t.co/CyP4LtCbiP']
search_term                   zipingpu dam
Name: 94, dtype: object
ID                     1016774079611797504
has_media                             True
medias         ['https://t.co/BA8THZOQwG']
search_term                   zipingpu dam
Name: 97, dtype: object
ID                     1060656032219840512
has_media                             True
medias         ['https://t.co/b1E3yOprAO']
search_term      yellowstone fires of 1988
Name: 104, dtype: object
ID                     1075261237246783488
has_media                             True
medias         ['https://t.co/DFSXZBSBNg']
search_term      yellowstone fires of 1988
Name: 109, dtype: object
ID                     1081830923023368192
has_media                             True
medias         ['https://t.co/gA1izMnxV0']
search_term             ocean surface wave
Name: 115, dtype: object
ID               

KeyboardInterrupt: 

In [ ]:
# For Extracting Location from 